26092022
Бруцкий-Стемпковский

v. 01

Изучение потоков и процессов. GIL.

Для получения списка ссылок используется модуль regex. Для работы с каталогами моуль os. Для работы с файлами из интернета модуль requests.

Требуется:
1. Реализовать с использованием потоков и процессов скачивание файлов из интернета. 
2. Список файлов для скачивания подготовить самостоятельно (например изображений, не менее 100 изображений или других объектов). 
3. Сравнить производительность с последовательным методом. 
4. Сравнивть производительность Thread и multiprocessing решений. Попробовать подобрать оптимальное число потоков/процессов. 

В первую очередь подотовим список ссылок для скачивания изображений.

В качестве подопытного выбран сайт ardom.by
Вообще нужен сайт, на котором могут быть собраны множество картинок в разделе типа "Портфолио"
На этом имеются изображения балконов.

В файле html.txt, содержащейся в корневом каталоге, находится исходный код сайта, содержащий ссылки на множество изображений. Откроем его и прочитаем в переменную code.

In [2]:
file_obj = open("html.txt")
code = file_obj.readlines()
file_obj.close()

len(code)

2708

В коде имеются ссылки на полноразмерное изображение и на миниатюры. Выберем последнее.
Типовая ссылка на такое изображение содержится, например, в строке 534 исходного кода.

In [3]:
code[534]

'\t\t\t<img alt="Обшивка балкона пластиком - заказать услуги в Минске" class="g-img__image" src="/thumb/2/IjkGpyv7FP-a0i78xcNIMQ/150r150/d/77-obshivka-sten-sirenevymi-plastikovymi-paneljami-pol-linoleum_3.jpg">\n'

Она начинается на сочетание символов '/thumb/' и заканчивается на '.jpg'
К ней нужно добавить префикс "ardom.by". Ссылка будет доступная для открытия.
Найдём их с помощью регулярных выражений.

In [4]:
import re
valid_string = re.compile(r"/thumb/(\S*?)jpg")

Создадим регулярное выражение и протестируем его.

In [5]:
ans = valid_string.findall(code[534])
if len(ans) == 1:
    print("ardom.by/thumb/" + ans[0] + "jpg")

ardom.by/thumb/2/IjkGpyv7FP-a0i78xcNIMQ/150r150/d/77-obshivka-sten-sirenevymi-plastikovymi-paneljami-pol-linoleum_3.jpg


Созададим список ссылок из исходного кода страницы.

In [6]:
list_of_links = []
for string_ in code:
    ans = valid_string.findall(string_)
    if len(ans) == 1:
        list_of_links.append("https://ardom.by/thumb/" + ans[0] + "jpg")

len(list_of_links)

200

Запишем ссылки в текстовый документ

In [7]:
file_obj = open("links.txt", "w")
[file_obj.write(list_of_links[_] + "\n") for _ in range(len(list_of_links))]
file_obj.close()

Теперь попробуем просто скачать картинки обычным способом, замерив при этом время.
Импортируем необхожимые модули для работы с каталогами и интернетом.

In [8]:
import os
import time
import requests
import shutil

print(os.getcwd())

d:\Works\Teach_ME_Skills\Материалы\ДЗ_Бруцкий-Стемпковский\DS3onl\Бруцкий-Стемпковский Николай\ДЗ_06_Бруцкий-Стемпковский


Создадим в текущем каталоге папку для изображений.

In [9]:
if not os.path.isdir("folder_for_images"):
     os.mkdir("folder_for_images")

Определим функцию-загрузчик.

In [10]:
path = os.getcwd() + "\\folder_for_images\\"

def saving_images(link):

    global path

    file_name = link.split("/")[-1]
    response = requests.get(url=link, timeout=10)

    if response.status_code == 200:
        open(path+file_name, "wb").write(response.content)

Скачаем изображения с GIL.

In [10]:
start = time.time()

for link in list_of_links:
    saving_images(link)

end = time.time()
print("Исходное время " + str(round(end - start, 3)) + " c")

Исходное время 60.836 c


Удалим полученную папку. Создадим ее вновь для следующего метода.

In [11]:
if os.path.isdir("folder_for_images"):
    shutil.rmtree(os.getcwd() + "\\folder_for_images")
if not os.path.isdir("folder_for_images"):
     os.mkdir("folder_for_images")

Скачаем изображения, управляя потоками с помощью библиотеки ThreadPoolExecutor.

In [12]:
import concurrent.futures

start = time.time()

with concurrent.futures.ThreadPoolExecutor(8) as executor:
    executor.map(saving_images, list_of_links)

end = time.time()
print("Время с ThreadPoolExecutor " + str(round(end - start, 3)) + " c")


Время с ThreadPoolExecutor 44.747 c


Снова удалим полученную папку. Создадим ее вновь для следующего метода.

In [13]:
if os.path.isdir("folder_for_images"):
    shutil.rmtree(os.getcwd() + "\\folder_for_images")
if not os.path.isdir("folder_for_images"):
     os.mkdir("folder_for_images")

Скачаем изображения, управляя процессами с помощью библиотеки Multiprocessing.

In [13]:
from multiprocessing import Pool
from functools import partial

start = time.time()

if __name__ == '__main__':
    p = Pool(4)
    saving_images_part = partial(saving_images, list_of_links)
    p.map(saving_images_part, list_of_links)

end = time.time()
print("Время с Multiprocessing " + str(round(end - start, 3)) + " c")

ImportError: cannot import name 'Pool' from 'multiprocessing' (d:\Works\Teach_ME_Skills\Материалы\ДЗ_Бруцкий-Стемпковский\DS3onl\Бруцкий-Стемпковский Николай\ДЗ_06_Бруцкий-Стемпковский\multiprocessing.py)